<a href="https://colab.research.google.com/github/patthub/Institute-of-Literary-Research/blob/master/Has%C5%82a_osobowe_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import _sqlite3
import pdb
import re as re
import PH_functions as PH
import numpy as np
import pprint
import xml.etree.ElementTree as ET
import json 
import urllib
print("done")



done


In [ ]:
pd.set_option('display.max_rows', None)
%load_ext google.colab.data_table
%load_ext rpy2.ipython


The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table
The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [ ]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [ ]:
%%R
install.packages("plyr")
install.packages("stringr")
install.packages("tidyverse")
install.packages("reshape2")
install.packages("zoo")
install.packages("splitstackshape")
install.packages("dplyr")
install.packages("svMisc")
install.packages("viafr")


In [ ]:
%%R
library(plyr)
library(stringr)
library(tidyverse)
library(reshape2)
library(zoo)
library(splitstackshape)
library(dplyr)
library(svMisc)
library(viafr)

In [ ]:
%%R
Mickiewicz_korpus_raw <- readLines("Mickiewicz_korpus_full.txt")
write.csv(Mickiewicz_korpus_raw, "Mickiewicz_korpus_raw.csv")


In [ ]:

#df = pd.read_csv("HO50-51.csv")
df = pd.read_csv("Mickiewicz_korpus_raw.csv")

In [ ]:
df["x"].replace('', np.nan, inplace=True)
df.dropna(subset=["x"], inplace=True)
df['digit'] = df["x"].str.contains("^\\d{1,4}\\.", regex=True)
df['row_id'] = np.arange(len(df))
df
def f(row):
    if row['digit'] == True:
        val = row['row_id']
        return val
df['id'] = df.apply(lambda x: f(x), axis = 1)
df
df['id'].ffill(axis=0, inplace=True)
df = df.groupby('id')['x'].apply(' '.join).reset_index()
df['x']
df.rename(columns={"x" : "Nazwa"}, inplace=True)


In [ ]:
df["numerCalostki"] = df["Nazwa"].str.extract('(^\d{1,4}\.)', expand=True)
df.loc[:, 'numerCalostki'] = df.loc[:, 'numerCalostki'].ffill()

In [ ]:
df["nawiasKw"] = df["Nazwa"].str.extract("(\\[[^\\]]*;.*\\])", expand=True)

In [ ]:
df["nawiasKw1"] = df["nawiasKw"].str.replace(";", "~")

In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace(str(df['nawiasKw'][i]), str(df['nawiasKw1'][i]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace("W;", "W:")


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
df["przedDzieleniem"] = df["Nazwa"].copy()

In [ ]:
df.to_csv("korpus_main.csv")

PARSER R SPLIT

In [ ]:
%%R
korpus_main <- read.csv("korpus_main.csv")


In [ ]:
%%R
###split by single work###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "\\.\\s{0,1}[—-]" )))
names(korpus_main)[1] <-"Nazwa"

###split by ;###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "(?=;)" )))
names(korpus_main)[1] <- "Nazwa"
###split by /odp###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "(?<=\\[?\\/odp\\.\\]?)" )))
names(korpus_main)[1] <-"Nazwa"

###split by /polem.###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "(?<=\\[?\\/polem\\.\\]?)" )))
names(korpus_main)[1] <-"Nazwa"

###split by /nawiąz.###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "(?<=\\[?nawiąz\\.\\]?)" )))
names(korpus_main)[1] <- "Nazwa"

###split by Rec.###

korpus_main <- as.data.frame(unlist(str_split(korpus_main$Nazwa, "(?<=Rec\\.\\:)" )))
names(korpus_main)[1] <- "Nazwa"


In [ ]:
%%R
write_csv(korpus_main, "data.csv")


In [ ]:
df = pd.read_csv("data.csv")

In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace("~", ";")

In [ ]:
df['tworca'] = df["Nazwa"].str.extract("([A-ZŚŁŻŹĆŃĘÓĄ1ÉŁ -]{3,100} [A-ZŚŁŻŹĆŃĘÓĄÉŁ ]{3,100})", expand=True)


In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace(str(df['tworca'][i]), "")


In [ ]:
df.loc[:, 'tworca'] = df.loc[:, 'tworca'].ffill()

SPLIT ;

In [ ]:
df["Pełen rekord"] = df["Nazwa"].copy()

In [ ]:
skroty = pd.read_csv("skroty.csv")
#skroty_test = pd.read_csv("skroty1.csv")
skroty_test = pd.read_csv("skroty1 - skroty1-kopia.csv")


In [ ]:
skroty_list = skroty['nazwa'].values.tolist()

skroty_list_test = skroty_test['pbl'].values.tolist()
skroty_list_test1 = str(skroty_list_test)

#skróty do pattern

pattern = ' | '.join(skroty_list)
#pattern_test =  (' | '.join((skroty_list_test)))
pattern_test =  (' | '.join((skroty_list_test)))

pattern_test1 = re.escape(pattern_test)
skroty_grupa = '(' + pattern_test + ')'


In [ ]:
tmp_df = df['Nazwa'].str.extract(skroty_grupa)

tmp_df = pd.DataFrame(tmp_df[0])
df['skroty'] = tmp_df
df['skroty'] = df['skroty'].str[1:]

In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace(str(df['skroty'][i]), "")

In [ ]:
df["numer"] = df["Nazwa"].str.extract('(nr\s.+)', expand=True)
#df["numer_czasopisma"] = df["Nazwa"].str.findall('(\d+\:\d+(\-\d+)?)|(\d*$)')
df["numer_czasopisma"] = df["Nazwa"].str.findall('(\d*[-—:]?\d*?[-—:]?\d*?\.?$)')
#df["strony_seria"] = df["Nazwa"].str.findall("(ss\s?\.?\s?[\d+\s,nlb tabl\. ilustr\. portr\. plany\. mapa\. mapy\.]*)")

#df["strony_seria"] = df["Nazwa"].str.findall("ss\..+")
df["strony"] = df["Nazwa"].str.findall("s\.\s?\d+\s?[-—]?\s?\d+")


In [ ]:
df["adnotacja"] = df["Nazwa"].str.extract('(\[[A-ZĄĆĘŁÓŚŻŹŃa-zzćęłóśżźąń\.\s\d;,]+\][\.,]?$)', expand=True)


In [ ]:
df.to_csv("df.csv")

In [ ]:
%%R
df <- read.csv("df.csv")
tmp_df1 <- str_split(df$Nazwa, "(?=Oprac)|(?=Ilustr)|(?=W opracowaniu)|(?=Wstęp:)|(?=Tłum)|(?=Z objaśnieniami)|(?=Przedm\\.)|(?=Przedm )|(?=Przedmowa)|(?=Wybór)|(?=Red\\.?)|(?=Przeł\\.)|(?=Przeł )|(?=Przełożył)|(?=Adapt\\.?)|(?=Aut\\.?)|(?=Fot\\.?)|(?=Oprac\\.graf\\.)", 2)
tmp_df1 <- as.data.frame(tmp_df1)
tmp_df1 <- t(tmp_df1)
tmp_df1<- as.data.frame(tmp_df1)
write.csv(tmp_df1, "tmp_df1.csv")

In [ ]:
tmp_df1 = pd.read_csv("tmp_df1.csv")
df["Autor"] = tmp_df1["V2"]

In [ ]:
%%R
df <- read.csv("df.csv")
tmp_df1 <- str_split(df$Nazwa, "(?=Wyd\\.\\s?\\d)|(?=Wydanie\\s?\\d)|(?=wyd\\.\\s?\\d)", 2)
tmp_df1 <- as.data.frame(tmp_df1)
tmp_df1 <- t(tmp_df1)
tmp_df1<- as.data.frame(tmp_df1)
write.csv(tmp_df1, "tmp_df1.csv")


In [ ]:
tmp_df1 = pd.read_csv("tmp_df1.csv")
df["Wydanie"] = tmp_df1["V2"]

In [ ]:
df.to_csv("df2.csv")

In [ ]:
%%R
df <- read.csv("df.csv")
tmp_df1 <- str_split(df$Nazwa, "(zob.)", 2)
tmp_df1 <- as.data.frame(tmp_df1)
tmp_df1 <- t(tmp_df1)
tmp_df1<- as.data.frame(tmp_df1)
write.csv(tmp_df1, "tmp_df1.csv")


In [ ]:
tmp_df1 = pd.read_csv("tmp_df1.csv")
df["Zobacz"] = tmp_df1["V2"]

In [ ]:
%%R
df <- read.csv("df.csv")
tmp_df1 <- str_split(df$Nazwa, "(?=Kraków)|(?=Warszawa)|(?=Katowice)|(?=Białystok)|(?=Poznań )|(?=Bydgoszcz)|(?=Łódź)|(?=Toruń)|(?=Gdańsk)|(?=Wrocław)|(?=Wwa)|(?=W-wa)|(?=Gdynia)|(?=Kat\\., Śl\\.,)|(?=Pozn)|(?=Lubl)|(?=Wrocł)|(?=Kr\\.)", 2)
tmp_df1 <- as.data.frame(tmp_df1)
tmp_df1 <- t(tmp_df1)
tmp_df1<- as.data.frame(tmp_df1)
write.csv(tmp_df1, "tmp_df1.csv")

In [ ]:
tmp_df1 = pd.read_csv("tmp_df1.csv")
df["Miejsce wydania"] = tmp_df1["V2"]

In [ ]:
for i in df.index:
  if df["Wydanie"][i] == df["Nazwa"][i]:
    df["Wydanie"][i] = ""
for i in df.index:
  if df["Autor"][i] == df["Nazwa"][i]:
    df["Autor"][i] = ""
for i in df.index:
  if df["Zobacz"][i] == df["Nazwa"][i]:
    df["Zobacz"][i] = ""
for i in df.index:
  if df["Miejsce wydania"][i] == df["Nazwa"][i]:
    df["Miejsce wydania"][i] = ""



In [ ]:
for i in df.index:
  df['Autor'][i] = str(df['Autor'][i]).replace(str(df['Wydanie'][i]), "")

In [ ]:
type(df["Wydanie"])

In [ ]:
df.loc[:, 'numer_czasopisma1'] = df.numer_czasopisma.map(lambda x: x[0])

In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace(str(df['numer_czasopisma1'][i]), "")

In [ ]:
df["Strony_seria"] = df["Wydanie"].str.extract("(ss\s?\.?\s?[\d+\s,nlb tabl\. ilustr\. portr\. plany\. mapa\. mapy\. nlb\.]*)")


In [ ]:
for i in df.index:
  df['Wydanie'][i] = str(df['Wydanie'][i]).replace(str(df['Strony_seria'][i]), "")

In [ ]:
df.head()

In [ ]:
df["numerCalostki"] = df["Nazwa"].str.extract('(^\d{1,4}\.?)', expand=True)
df.loc[:, 'numerCalostki'] = df.loc[:, 'numerCalostki'].ffill()

In [ ]:
for i in df.index:
  df['Nazwa'][i] = df['Nazwa'][i].replace(str(df['numerCalostki'][i]), "")

In [ ]:
for i in df.index:
  df['Miejsce wydania'][i] = str(df['Miejsce wydania'][i]).replace(str(df['Strony_seria'][i]), "")

In [ ]:
for i in df.index:
  df['Wydanie'][i] = df['Wydanie'][i].replace(str(df['Miejsce wydania'][i]), "")

In [ ]:
df['Tytuł'] = df['Nazwa'].str.lstrip(";")

In [ ]:
df['Tytuł'] = df['Tytuł'].str.strip()

In [ ]:
df['Autor1'] = df["Tytuł"].str.extract('([-ZŚŁŻŹĆŃĘÓĄ1ÉŁ]{1}[a-zśłżźćńęóą1éł]+\s[-ZŚŁŻŹĆŃĘÓĄ1ÉŁ]{1}\.\s?)', expand = True)


In [ ]:
for i in df.index:
  df['Tytuł'][i] = df['Tytuł'][i].replace(str(df['Autor1'][i]), "")

In [ ]:
for i in df.index:
  df['Tytuł'][i] = df['Tytuł'][i].replace(str(df['Autor'][i]), "")

In [ ]:
for i in df.index:
  df['Tytuł'][i] = df['Tytuł'][i].replace(str(df['numer'][i]), "")

In [ ]:
for i in df.index:
  df['Tytuł'][i] = df['Tytuł'][i].replace(str(df['Miejsce wydania'][i]), "")

In [ ]:
for i in df.index:
  df['Tytuł'][i] = df['Tytuł'][i].replace(str(df['Strony_seria'][i]), "")

In [ ]:
for i in df.index:
  df['numer'][i] = str(df['numer'][i]).replace(str(df['adnotacja'][i]), "")

In [ ]:
df_marc = pd.DataFrame()

In [ ]:
df_marc['600'] = '$a' + df['tworca'].astype(str)

In [ ]:
df_marc.head()

In [ ]:
df.to_csv("df2.csv")

In [ ]:
df.head(25)

In [ ]:
pd.isnull(df['adnotacja'].iloc[0])

True

True